In [1]:
import numpy as np
import pandas as pd

from sklearn.ensemble import RandomForestClassifier
from catboost import CatBoostClassifier

from sklearn.metrics import roc_auc_score, accuracy_score, roc_curve
from warnings import filterwarnings
filterwarnings('ignore')

In [2]:
train_df = pd.read_csv("../data/train_folds.csv")

In [3]:
train_df.shape

(692, 15)

In [4]:
train_df.head()

,Gender,Married,Dependents,Education,Self_Employed,LoanAmount,Loan_Amount_Term,Credit_History,Total_Income,Semiurban,Urban,Loan_Status,total_income_by_loanAmt,Loan_Amount_Term_By_amt,kfold
0,1.0,1.0,0.000000,1.0,0.0,104.000000,360.000000,1.0,3819.000000,0.000000,1.0,1,1348.443140,0.288889,0
1,1.0,0.0,1.000000,1.0,0.0,117.000000,360.000000,1.0,6177.000000,0.000000,1.0,1,2787.298488,0.325000,0
2,1.0,1.0,2.419781,1.0,0.0,182.901096,334.813149,1.0,7142.630796,0.419781,0.0,0,1525.048970,0.546278,0
3,1.0,1.0,0.000000,1.0,0.0,143.701687,228.983805,0.0,7818.452295,0.000000,0.0,0,2960.178157,0.627563,0
4,1.0,1.0,0.000000,1.0,0.0,100.000000,360.000000,1.0,3539.000000,0.000000,1.0,1,1252.452100,0.277778,0


In [5]:
targets = train_df.Loan_Status.values

In [6]:
def train_fold(df, model, fold):
    train = df.loc[df.kfold!=fold]
    valid = df.loc[df.kfold==fold]
    
    X_train = train.drop(["kfold", "Loan_Status"],axis=1).values
    X_valid = valid.drop(["kfold", "Loan_Status"],axis=1).values
    
    y_train = train.Loan_Status.values
    y_valid = valid.Loan_Status.values
    
    model.fit(X_train, y_train)
    preds = model.predict(X_valid)
    acc = accuracy_score(y_valid, preds)
    roc = roc_auc_score(y_valid, preds)
    return acc, roc

In [7]:
def run_training(df, model):
    accs = []
    rocs = []
    for i in range(5):
        acc, roc = train_fold(df, model, i)
        accs.append(acc)
        rocs.append(roc)
    return accs, rocs

In [8]:
best_params_rf = {'criterion': 'gini', 'max_depth': None, 'n_estimators': 300}

In [10]:
rf_model = RandomForestClassifier(**best_params_rf, n_jobs=-1)

In [9]:
cat_model = CatBoostClassifier()

In [11]:
rf_accs, rf_rocs = run_training(train_df, rf_model)

In [12]:
cat_accs, cat_rocs = run_training(train_df, cat_model)

Learning rate set to 0.008
0:	learn: 0.6879598	total: 198ms	remaining: 3m 18s
1:	learn: 0.6841813	total: 206ms	remaining: 1m 42s
2:	learn: 0.6804470	total: 214ms	remaining: 1m 11s
3:	learn: 0.6757762	total: 222ms	remaining: 55.4s
4:	learn: 0.6715496	total: 230ms	remaining: 45.8s
5:	learn: 0.6678469	total: 234ms	remaining: 38.7s
6:	learn: 0.6638415	total: 241ms	remaining: 34.2s
7:	learn: 0.6599452	total: 250ms	remaining: 31s
8:	learn: 0.6558427	total: 258ms	remaining: 28.4s
9:	learn: 0.6509365	total: 266ms	remaining: 26.3s
10:	learn: 0.6472089	total: 275ms	remaining: 24.7s
11:	learn: 0.6434381	total: 282ms	remaining: 23.2s
12:	learn: 0.6400649	total: 292ms	remaining: 22.1s
13:	learn: 0.6359038	total: 299ms	remaining: 21.1s
14:	learn: 0.6321974	total: 308ms	remaining: 20.2s
15:	learn: 0.6284953	total: 316ms	remaining: 19.4s
16:	learn: 0.6248290	total: 323ms	remaining: 18.7s
17:	learn: 0.6210146	total: 330ms	remaining: 18s
18:	learn: 0.6176163	total: 337ms	remaining: 17.4s
19:	learn: 0.61

181:	learn: 0.3786238	total: 1.47s	remaining: 6.6s
182:	learn: 0.3779821	total: 1.48s	remaining: 6.59s
183:	learn: 0.3774772	total: 1.48s	remaining: 6.58s
184:	learn: 0.3768659	total: 1.49s	remaining: 6.56s
185:	learn: 0.3760257	total: 1.5s	remaining: 6.54s
186:	learn: 0.3752581	total: 1.5s	remaining: 6.53s
187:	learn: 0.3747685	total: 1.51s	remaining: 6.52s
188:	learn: 0.3741399	total: 1.52s	remaining: 6.5s
189:	learn: 0.3737709	total: 1.52s	remaining: 6.5s
190:	learn: 0.3731482	total: 1.53s	remaining: 6.49s
191:	learn: 0.3726516	total: 1.54s	remaining: 6.48s
192:	learn: 0.3721006	total: 1.55s	remaining: 6.47s
193:	learn: 0.3716292	total: 1.55s	remaining: 6.46s
194:	learn: 0.3709488	total: 1.56s	remaining: 6.44s
195:	learn: 0.3703675	total: 1.57s	remaining: 6.43s
196:	learn: 0.3700655	total: 1.57s	remaining: 6.42s
197:	learn: 0.3694252	total: 1.58s	remaining: 6.41s
198:	learn: 0.3690079	total: 1.59s	remaining: 6.39s
199:	learn: 0.3685320	total: 1.6s	remaining: 6.38s
200:	learn: 0.3680

354:	learn: 0.3020118	total: 2.49s	remaining: 4.53s
355:	learn: 0.3018451	total: 2.5s	remaining: 4.51s
356:	learn: 0.3015597	total: 2.5s	remaining: 4.5s
357:	learn: 0.3012394	total: 2.5s	remaining: 4.49s
358:	learn: 0.3008672	total: 2.51s	remaining: 4.48s
359:	learn: 0.3005747	total: 2.51s	remaining: 4.47s
360:	learn: 0.3001789	total: 2.52s	remaining: 4.46s
361:	learn: 0.2999772	total: 2.52s	remaining: 4.45s
362:	learn: 0.2995608	total: 2.53s	remaining: 4.44s
363:	learn: 0.2991660	total: 2.53s	remaining: 4.43s
364:	learn: 0.2988131	total: 2.54s	remaining: 4.42s
365:	learn: 0.2984407	total: 2.54s	remaining: 4.41s
366:	learn: 0.2981676	total: 2.55s	remaining: 4.39s
367:	learn: 0.2978257	total: 2.55s	remaining: 4.38s
368:	learn: 0.2974364	total: 2.56s	remaining: 4.37s
369:	learn: 0.2972346	total: 2.56s	remaining: 4.36s
370:	learn: 0.2969220	total: 2.57s	remaining: 4.35s
371:	learn: 0.2965881	total: 2.57s	remaining: 4.34s
372:	learn: 0.2962124	total: 2.58s	remaining: 4.33s
373:	learn: 0.29

537:	learn: 0.2503964	total: 3.35s	remaining: 2.88s
538:	learn: 0.2500934	total: 3.36s	remaining: 2.87s
539:	learn: 0.2497947	total: 3.36s	remaining: 2.86s
540:	learn: 0.2496021	total: 3.37s	remaining: 2.85s
541:	learn: 0.2493268	total: 3.37s	remaining: 2.85s
542:	learn: 0.2490427	total: 3.37s	remaining: 2.84s
543:	learn: 0.2488303	total: 3.38s	remaining: 2.83s
544:	learn: 0.2486661	total: 3.38s	remaining: 2.82s
545:	learn: 0.2484669	total: 3.39s	remaining: 2.82s
546:	learn: 0.2482662	total: 3.39s	remaining: 2.81s
547:	learn: 0.2481018	total: 3.4s	remaining: 2.8s
548:	learn: 0.2480629	total: 3.4s	remaining: 2.79s
549:	learn: 0.2477474	total: 3.4s	remaining: 2.78s
550:	learn: 0.2474980	total: 3.41s	remaining: 2.78s
551:	learn: 0.2473001	total: 3.41s	remaining: 2.77s
552:	learn: 0.2470852	total: 3.42s	remaining: 2.76s
553:	learn: 0.2468518	total: 3.42s	remaining: 2.75s
554:	learn: 0.2466548	total: 3.42s	remaining: 2.75s
555:	learn: 0.2464138	total: 3.43s	remaining: 2.74s
556:	learn: 0.24

702:	learn: 0.2165807	total: 4.06s	remaining: 1.72s
703:	learn: 0.2162687	total: 4.07s	remaining: 1.71s
704:	learn: 0.2160381	total: 4.07s	remaining: 1.7s
705:	learn: 0.2158420	total: 4.08s	remaining: 1.7s
706:	learn: 0.2156997	total: 4.08s	remaining: 1.69s
707:	learn: 0.2154884	total: 4.08s	remaining: 1.69s
708:	learn: 0.2150668	total: 4.09s	remaining: 1.68s
709:	learn: 0.2148461	total: 4.09s	remaining: 1.67s
710:	learn: 0.2146503	total: 4.1s	remaining: 1.67s
711:	learn: 0.2143804	total: 4.1s	remaining: 1.66s
712:	learn: 0.2141755	total: 4.11s	remaining: 1.65s
713:	learn: 0.2139226	total: 4.11s	remaining: 1.65s
714:	learn: 0.2138065	total: 4.12s	remaining: 1.64s
715:	learn: 0.2137220	total: 4.12s	remaining: 1.63s
716:	learn: 0.2135580	total: 4.12s	remaining: 1.63s
717:	learn: 0.2132067	total: 4.13s	remaining: 1.62s
718:	learn: 0.2129988	total: 4.13s	remaining: 1.61s
719:	learn: 0.2127662	total: 4.14s	remaining: 1.61s
720:	learn: 0.2125112	total: 4.14s	remaining: 1.6s
721:	learn: 0.212

...

199:	learn: 0.3949830	total: 882ms	remaining: 3.53s
200:	learn: 0.3945877	total: 886ms	remaining: 3.52s
201:	learn: 0.3940630	total: 890ms	remaining: 3.52s
202:	learn: 0.3936523	total: 894ms	remaining: 3.51s
203:	learn: 0.3929218	total: 899ms	remaining: 3.51s
204:	learn: 0.3921156	total: 903ms	remaining: 3.5s
205:	learn: 0.3916336	total: 908ms	remaining: 3.5s
206:	learn: 0.3909747	total: 912ms	remaining: 3.5s
207:	learn: 0.3904728	total: 917ms	remaining: 3.49s
208:	learn: 0.3898435	total: 921ms	remaining: 3.49s
209:	learn: 0.3896193	total: 924ms	remaining: 3.48s
210:	learn: 0.3889258	total: 929ms	remaining: 3.47s
211:	learn: 0.3882915	total: 934ms	remaining: 3.47s
212:	learn: 0.3876488	total: 939ms	remaining: 3.47s
213:	learn: 0.3871998	total: 944ms	remaining: 3.47s
214:	learn: 0.3866090	total: 950ms	remaining: 3.47s
215:	learn: 0.3859239	total: 954ms	remaining: 3.46s
216:	learn: 0.3852385	total: 960ms	remaining: 3.46s
217:	learn: 0.3845970	total: 966ms	remaining: 3.46s
218:	learn: 0.3

382:	learn: 0.3160015	total: 1.95s	remaining: 3.14s
383:	learn: 0.3156615	total: 1.96s	remaining: 3.14s
384:	learn: 0.3153287	total: 1.96s	remaining: 3.13s
385:	learn: 0.3151701	total: 1.97s	remaining: 3.13s
386:	learn: 0.3146855	total: 1.97s	remaining: 3.13s
387:	learn: 0.3144232	total: 1.98s	remaining: 3.12s
388:	learn: 0.3141704	total: 1.99s	remaining: 3.12s
389:	learn: 0.3138307	total: 1.99s	remaining: 3.12s
390:	learn: 0.3135982	total: 2s	remaining: 3.12s
391:	learn: 0.3132837	total: 2.01s	remaining: 3.11s
392:	learn: 0.3130216	total: 2.01s	remaining: 3.11s
393:	learn: 0.3127558	total: 2.02s	remaining: 3.11s
394:	learn: 0.3125390	total: 2.03s	remaining: 3.1s
395:	learn: 0.3121898	total: 2.03s	remaining: 3.1s
396:	learn: 0.3120360	total: 2.04s	remaining: 3.1s
397:	learn: 0.3116977	total: 2.05s	remaining: 3.1s
398:	learn: 0.3115313	total: 2.05s	remaining: 3.09s
399:	learn: 0.3112685	total: 2.06s	remaining: 3.09s
400:	learn: 0.3109638	total: 2.07s	remaining: 3.09s
401:	learn: 0.31061

556:	learn: 0.2674947	total: 3.16s	remaining: 2.52s
557:	learn: 0.2672909	total: 3.17s	remaining: 2.51s
558:	learn: 0.2670879	total: 3.18s	remaining: 2.51s
559:	learn: 0.2668648	total: 3.18s	remaining: 2.5s
560:	learn: 0.2667109	total: 3.19s	remaining: 2.5s
561:	learn: 0.2665317	total: 3.2s	remaining: 2.49s
562:	learn: 0.2663344	total: 3.21s	remaining: 2.49s
563:	learn: 0.2660929	total: 3.21s	remaining: 2.48s
564:	learn: 0.2658427	total: 3.22s	remaining: 2.48s
565:	learn: 0.2656172	total: 3.23s	remaining: 2.48s
566:	learn: 0.2652450	total: 3.24s	remaining: 2.47s
567:	learn: 0.2650325	total: 3.25s	remaining: 2.47s
568:	learn: 0.2647034	total: 3.25s	remaining: 2.46s
569:	learn: 0.2643792	total: 3.26s	remaining: 2.46s
570:	learn: 0.2641617	total: 3.27s	remaining: 2.45s
571:	learn: 0.2639013	total: 3.27s	remaining: 2.45s
572:	learn: 0.2637445	total: 3.28s	remaining: 2.44s
573:	learn: 0.2634946	total: 3.29s	remaining: 2.44s
574:	learn: 0.2632529	total: 3.29s	remaining: 2.43s
575:	learn: 0.2

718:	learn: 0.2317962	total: 4.24s	remaining: 1.66s
719:	learn: 0.2315655	total: 4.25s	remaining: 1.65s
720:	learn: 0.2314278	total: 4.25s	remaining: 1.65s
721:	learn: 0.2312900	total: 4.26s	remaining: 1.64s
722:	learn: 0.2308925	total: 4.27s	remaining: 1.63s
723:	learn: 0.2306853	total: 4.27s	remaining: 1.63s
724:	learn: 0.2305975	total: 4.28s	remaining: 1.62s
725:	learn: 0.2304229	total: 4.29s	remaining: 1.62s
726:	learn: 0.2302863	total: 4.3s	remaining: 1.61s
727:	learn: 0.2301272	total: 4.3s	remaining: 1.61s
728:	learn: 0.2298925	total: 4.31s	remaining: 1.6s
729:	learn: 0.2296844	total: 4.32s	remaining: 1.6s
730:	learn: 0.2294601	total: 4.32s	remaining: 1.59s
731:	learn: 0.2293714	total: 4.33s	remaining: 1.58s
732:	learn: 0.2292154	total: 4.34s	remaining: 1.58s
733:	learn: 0.2290666	total: 4.34s	remaining: 1.57s
734:	learn: 0.2289433	total: 4.35s	remaining: 1.57s
735:	learn: 0.2287070	total: 4.36s	remaining: 1.56s
736:	learn: 0.2284674	total: 4.36s	remaining: 1.56s
737:	learn: 0.22

880:	learn: 0.2027908	total: 5.27s	remaining: 712ms
881:	learn: 0.2026459	total: 5.28s	remaining: 706ms
882:	learn: 0.2024426	total: 5.28s	remaining: 700ms
883:	learn: 0.2022768	total: 5.29s	remaining: 694ms
884:	learn: 0.2021302	total: 5.29s	remaining: 688ms
885:	learn: 0.2020025	total: 5.3s	remaining: 682ms
886:	learn: 0.2018316	total: 5.31s	remaining: 676ms
887:	learn: 0.2016644	total: 5.31s	remaining: 670ms
888:	learn: 0.2015386	total: 5.32s	remaining: 664ms
889:	learn: 0.2014053	total: 5.33s	remaining: 658ms
890:	learn: 0.2012174	total: 5.33s	remaining: 653ms
891:	learn: 0.2010822	total: 5.34s	remaining: 647ms
892:	learn: 0.2009610	total: 5.35s	remaining: 641ms
893:	learn: 0.2007982	total: 5.35s	remaining: 635ms
894:	learn: 0.2006415	total: 5.36s	remaining: 629ms
895:	learn: 0.2004467	total: 5.37s	remaining: 623ms
896:	learn: 0.2003215	total: 5.37s	remaining: 617ms
897:	learn: 0.2001833	total: 5.38s	remaining: 611ms
898:	learn: 0.2000588	total: 5.38s	remaining: 605ms
899:	learn: 0

In [13]:
np.mean(rf_accs)

0.8584402043582525

In [14]:
np.mean(cat_accs)

0.8541236575956626

In [15]:
def predict_probs(df, model, name, fold):
    df_train = df[df.kfold != fold].reset_index(drop=True)
    df_valid = df[df.kfold == fold].reset_index(drop=True)

    X_train = df_train.drop(["Loan_Status"], axis=1).values
    X_valid = df_valid.drop(["Loan_Status"], axis=1).values

    y_train = df_train.Loan_Status.values
    y_valid = df_valid.Loan_Status.values
    
    model.fit(X_train, y_train)

    pred = model.predict_proba(X_valid)[:, 1]
    
    df_valid.loc[:, name] = pred
    
    return df_valid

In [16]:
def run_training(df, model, name):
    dfs = []
    for i in range(5):
        temp_df = predict_probs(df, model, name, i)
        dfs.append(temp_df)

    fin_df = pd.concat(dfs)
    return fin_df

In [17]:
models = [rf_model, cat_model]

In [18]:
fin_df = train_df.copy()

In [19]:
for mod,name in zip(models, ['rf_probs', 'cat_probs']):
    fin_df = run_training(fin_df, mod, name)

Learning rate set to 0.008
0:	learn: 0.6867052	total: 6.16ms	remaining: 6.16s
1:	learn: 0.6803668	total: 11.8ms	remaining: 5.89s
2:	learn: 0.6742709	total: 17.7ms	remaining: 5.89s
3:	learn: 0.6677251	total: 24.3ms	remaining: 6.04s
4:	learn: 0.6619605	total: 30.6ms	remaining: 6.1s
5:	learn: 0.6549410	total: 36.3ms	remaining: 6.02s
6:	learn: 0.6489098	total: 42.3ms	remaining: 6s
7:	learn: 0.6436744	total: 48.2ms	remaining: 5.98s
8:	learn: 0.6376391	total: 54.1ms	remaining: 5.96s
9:	learn: 0.6325830	total: 60.1ms	remaining: 5.95s
10:	learn: 0.6265051	total: 65.8ms	remaining: 5.92s
11:	learn: 0.6210270	total: 71.6ms	remaining: 5.89s
12:	learn: 0.6157816	total: 77.4ms	remaining: 5.88s
13:	learn: 0.6104278	total: 83.1ms	remaining: 5.85s
14:	learn: 0.6053208	total: 88.8ms	remaining: 5.83s
15:	learn: 0.6009948	total: 95ms	remaining: 5.84s
16:	learn: 0.5959295	total: 101ms	remaining: 5.82s
17:	learn: 0.5914387	total: 106ms	remaining: 5.79s
18:	learn: 0.5869115	total: 112ms	remaining: 5.78s
19:	

179:	learn: 0.3021970	total: 1.08s	remaining: 4.94s
180:	learn: 0.3016887	total: 1.09s	remaining: 4.94s
181:	learn: 0.3010010	total: 1.1s	remaining: 4.93s
182:	learn: 0.3004362	total: 1.1s	remaining: 4.92s
183:	learn: 0.2997003	total: 1.11s	remaining: 4.92s
184:	learn: 0.2990190	total: 1.11s	remaining: 4.91s
185:	learn: 0.2982927	total: 1.12s	remaining: 4.91s
186:	learn: 0.2978086	total: 1.13s	remaining: 4.91s
187:	learn: 0.2973033	total: 1.13s	remaining: 4.9s
188:	learn: 0.2968566	total: 1.14s	remaining: 4.89s
189:	learn: 0.2961416	total: 1.15s	remaining: 4.89s
190:	learn: 0.2957647	total: 1.15s	remaining: 4.88s
191:	learn: 0.2952313	total: 1.16s	remaining: 4.88s
192:	learn: 0.2945013	total: 1.16s	remaining: 4.87s
193:	learn: 0.2937543	total: 1.17s	remaining: 4.86s
194:	learn: 0.2934533	total: 1.17s	remaining: 4.85s
195:	learn: 0.2928643	total: 1.18s	remaining: 4.84s
196:	learn: 0.2923285	total: 1.19s	remaining: 4.84s
197:	learn: 0.2919002	total: 1.19s	remaining: 4.83s
198:	learn: 0.2

343:	learn: 0.2357047	total: 2.19s	remaining: 4.18s
344:	learn: 0.2355275	total: 2.2s	remaining: 4.17s
345:	learn: 0.2353011	total: 2.2s	remaining: 4.17s
346:	learn: 0.2349078	total: 2.21s	remaining: 4.16s
347:	learn: 0.2344315	total: 2.22s	remaining: 4.16s
348:	learn: 0.2341740	total: 2.23s	remaining: 4.15s
349:	learn: 0.2339089	total: 2.23s	remaining: 4.15s
350:	learn: 0.2336879	total: 2.24s	remaining: 4.14s
351:	learn: 0.2334381	total: 2.25s	remaining: 4.14s
352:	learn: 0.2331700	total: 2.25s	remaining: 4.13s
353:	learn: 0.2330377	total: 2.26s	remaining: 4.13s
354:	learn: 0.2324912	total: 2.27s	remaining: 4.12s
355:	learn: 0.2322233	total: 2.27s	remaining: 4.11s
356:	learn: 0.2320156	total: 2.28s	remaining: 4.11s
357:	learn: 0.2317939	total: 2.29s	remaining: 4.1s
358:	learn: 0.2317010	total: 2.29s	remaining: 4.09s
359:	learn: 0.2313546	total: 2.3s	remaining: 4.09s
360:	learn: 0.2310125	total: 2.31s	remaining: 4.08s
361:	learn: 0.2307533	total: 2.31s	remaining: 4.08s
362:	learn: 0.23

508:	learn: 0.1942795	total: 3.28s	remaining: 3.17s
509:	learn: 0.1940871	total: 3.29s	remaining: 3.16s
510:	learn: 0.1939120	total: 3.29s	remaining: 3.15s
511:	learn: 0.1937803	total: 3.3s	remaining: 3.15s
512:	learn: 0.1936140	total: 3.31s	remaining: 3.14s
513:	learn: 0.1933439	total: 3.31s	remaining: 3.13s
514:	learn: 0.1930629	total: 3.32s	remaining: 3.13s
515:	learn: 0.1927527	total: 3.33s	remaining: 3.12s
516:	learn: 0.1924758	total: 3.33s	remaining: 3.11s
517:	learn: 0.1922960	total: 3.34s	remaining: 3.11s
518:	learn: 0.1921312	total: 3.34s	remaining: 3.1s
519:	learn: 0.1919437	total: 3.35s	remaining: 3.09s
520:	learn: 0.1917211	total: 3.35s	remaining: 3.08s
521:	learn: 0.1915923	total: 3.36s	remaining: 3.08s
522:	learn: 0.1913588	total: 3.37s	remaining: 3.07s
523:	learn: 0.1910123	total: 3.37s	remaining: 3.06s
524:	learn: 0.1908160	total: 3.38s	remaining: 3.06s
525:	learn: 0.1905664	total: 3.38s	remaining: 3.05s
526:	learn: 0.1902911	total: 3.39s	remaining: 3.04s
527:	learn: 0.

693:	learn: 0.1584048	total: 4.38s	remaining: 1.93s
694:	learn: 0.1581708	total: 4.38s	remaining: 1.92s
695:	learn: 0.1579935	total: 4.39s	remaining: 1.92s
696:	learn: 0.1578180	total: 4.39s	remaining: 1.91s
697:	learn: 0.1576429	total: 4.4s	remaining: 1.9s
698:	learn: 0.1575244	total: 4.41s	remaining: 1.9s
699:	learn: 0.1574279	total: 4.41s	remaining: 1.89s
700:	learn: 0.1571551	total: 4.42s	remaining: 1.89s
701:	learn: 0.1568794	total: 4.42s	remaining: 1.88s
702:	learn: 0.1567876	total: 4.43s	remaining: 1.87s
703:	learn: 0.1567048	total: 4.44s	remaining: 1.86s
704:	learn: 0.1564608	total: 4.44s	remaining: 1.86s
705:	learn: 0.1562096	total: 4.45s	remaining: 1.85s
706:	learn: 0.1559739	total: 4.45s	remaining: 1.85s
707:	learn: 0.1558575	total: 4.46s	remaining: 1.84s
708:	learn: 0.1556803	total: 4.47s	remaining: 1.83s
709:	learn: 0.1555850	total: 4.47s	remaining: 1.83s
710:	learn: 0.1554826	total: 4.48s	remaining: 1.82s
711:	learn: 0.1553586	total: 4.48s	remaining: 1.81s
712:	learn: 0.1

...

189:	learn: 0.3197933	total: 957ms	remaining: 4.08s
190:	learn: 0.3193478	total: 962ms	remaining: 4.08s
191:	learn: 0.3189872	total: 968ms	remaining: 4.07s
192:	learn: 0.3185948	total: 973ms	remaining: 4.07s
193:	learn: 0.3180011	total: 979ms	remaining: 4.07s
194:	learn: 0.3171461	total: 985ms	remaining: 4.06s
195:	learn: 0.3164404	total: 990ms	remaining: 4.06s
196:	learn: 0.3160793	total: 996ms	remaining: 4.06s
197:	learn: 0.3155095	total: 1s	remaining: 4.06s
198:	learn: 0.3150094	total: 1.01s	remaining: 4.05s
199:	learn: 0.3144681	total: 1.01s	remaining: 4.05s
200:	learn: 0.3137267	total: 1.02s	remaining: 4.05s
201:	learn: 0.3130902	total: 1.02s	remaining: 4.04s
202:	learn: 0.3126229	total: 1.03s	remaining: 4.04s
203:	learn: 0.3121339	total: 1.03s	remaining: 4.03s
204:	learn: 0.3117326	total: 1.04s	remaining: 4.02s
205:	learn: 0.3113682	total: 1.04s	remaining: 4.02s
206:	learn: 0.3106290	total: 1.05s	remaining: 4.01s
207:	learn: 0.3102513	total: 1.05s	remaining: 4s
208:	learn: 0.3097

352:	learn: 0.2571224	total: 1.99s	remaining: 3.64s
353:	learn: 0.2569299	total: 1.99s	remaining: 3.63s
354:	learn: 0.2567711	total: 2s	remaining: 3.63s
355:	learn: 0.2566911	total: 2s	remaining: 3.63s
356:	learn: 0.2563816	total: 2.01s	remaining: 3.62s
357:	learn: 0.2561940	total: 2.02s	remaining: 3.62s
358:	learn: 0.2559763	total: 2.02s	remaining: 3.62s
359:	learn: 0.2553879	total: 2.03s	remaining: 3.61s
360:	learn: 0.2551938	total: 2.04s	remaining: 3.61s
361:	learn: 0.2547664	total: 2.05s	remaining: 3.61s
362:	learn: 0.2545515	total: 2.05s	remaining: 3.6s
363:	learn: 0.2543549	total: 2.06s	remaining: 3.6s
364:	learn: 0.2540962	total: 2.07s	remaining: 3.6s
365:	learn: 0.2537205	total: 2.07s	remaining: 3.59s
366:	learn: 0.2536344	total: 2.08s	remaining: 3.59s
367:	learn: 0.2534071	total: 2.09s	remaining: 3.58s
368:	learn: 0.2528949	total: 2.09s	remaining: 3.58s
369:	learn: 0.2527429	total: 2.1s	remaining: 3.57s
370:	learn: 0.2524285	total: 2.1s	remaining: 3.57s
371:	learn: 0.2519060	t

511:	learn: 0.2168941	total: 3.05s	remaining: 2.91s
512:	learn: 0.2167109	total: 3.06s	remaining: 2.9s
513:	learn: 0.2164536	total: 3.06s	remaining: 2.9s
514:	learn: 0.2160361	total: 3.07s	remaining: 2.89s
515:	learn: 0.2159846	total: 3.08s	remaining: 2.89s
516:	learn: 0.2157526	total: 3.08s	remaining: 2.88s
517:	learn: 0.2156095	total: 3.09s	remaining: 2.88s
518:	learn: 0.2153801	total: 3.1s	remaining: 2.87s
519:	learn: 0.2152344	total: 3.11s	remaining: 2.87s
520:	learn: 0.2150077	total: 3.11s	remaining: 2.86s
521:	learn: 0.2147731	total: 3.12s	remaining: 2.86s
522:	learn: 0.2144766	total: 3.13s	remaining: 2.85s
523:	learn: 0.2142615	total: 3.13s	remaining: 2.85s
524:	learn: 0.2140437	total: 3.14s	remaining: 2.84s
525:	learn: 0.2138254	total: 3.15s	remaining: 2.83s
526:	learn: 0.2135568	total: 3.15s	remaining: 2.83s
527:	learn: 0.2134746	total: 3.16s	remaining: 2.82s
528:	learn: 0.2133600	total: 3.16s	remaining: 2.82s
529:	learn: 0.2132503	total: 3.17s	remaining: 2.81s
530:	learn: 0.2

687:	learn: 0.1818735	total: 4.17s	remaining: 1.89s
688:	learn: 0.1816809	total: 4.18s	remaining: 1.89s
689:	learn: 0.1814648	total: 4.19s	remaining: 1.88s
690:	learn: 0.1810972	total: 4.19s	remaining: 1.88s
691:	learn: 0.1808438	total: 4.2s	remaining: 1.87s
692:	learn: 0.1805803	total: 4.21s	remaining: 1.86s
693:	learn: 0.1803804	total: 4.21s	remaining: 1.86s
694:	learn: 0.1802301	total: 4.22s	remaining: 1.85s
695:	learn: 0.1800072	total: 4.22s	remaining: 1.84s
696:	learn: 0.1799037	total: 4.23s	remaining: 1.84s
697:	learn: 0.1797510	total: 4.24s	remaining: 1.83s
698:	learn: 0.1795017	total: 4.24s	remaining: 1.83s
699:	learn: 0.1792794	total: 4.25s	remaining: 1.82s
700:	learn: 0.1790839	total: 4.25s	remaining: 1.81s
701:	learn: 0.1789479	total: 4.26s	remaining: 1.81s
702:	learn: 0.1787780	total: 4.27s	remaining: 1.8s
703:	learn: 0.1785573	total: 4.27s	remaining: 1.8s
704:	learn: 0.1783839	total: 4.28s	remaining: 1.79s
705:	learn: 0.1782029	total: 4.28s	remaining: 1.78s
706:	learn: 0.1

854:	learn: 0.1534282	total: 5.23s	remaining: 887ms
855:	learn: 0.1533076	total: 5.24s	remaining: 881ms
856:	learn: 0.1532133	total: 5.24s	remaining: 875ms
857:	learn: 0.1531529	total: 5.25s	remaining: 869ms
858:	learn: 0.1530219	total: 5.26s	remaining: 863ms
859:	learn: 0.1529059	total: 5.26s	remaining: 857ms
860:	learn: 0.1527602	total: 5.27s	remaining: 851ms
861:	learn: 0.1526317	total: 5.28s	remaining: 845ms
862:	learn: 0.1524486	total: 5.29s	remaining: 839ms
863:	learn: 0.1523665	total: 5.29s	remaining: 833ms
864:	learn: 0.1522818	total: 5.3s	remaining: 827ms
865:	learn: 0.1521014	total: 5.31s	remaining: 821ms
866:	learn: 0.1519574	total: 5.31s	remaining: 815ms
867:	learn: 0.1516916	total: 5.32s	remaining: 809ms
868:	learn: 0.1515285	total: 5.33s	remaining: 803ms
869:	learn: 0.1513538	total: 5.33s	remaining: 797ms
870:	learn: 0.1512038	total: 5.34s	remaining: 791ms
871:	learn: 0.1510635	total: 5.34s	remaining: 785ms
872:	learn: 0.1508645	total: 5.35s	remaining: 779ms
873:	learn: 0

In [20]:
fin_df.head()

,Gender,Married,Dependents,Education,Self_Employed,LoanAmount,Loan_Amount_Term,Credit_History,Total_Income,Semiurban,Urban,Loan_Status,total_income_by_loanAmt,Loan_Amount_Term_By_amt,kfold,rf_probs,cat_probs
0,1.0,1.0,0.000000,1.0,0.0,104.000000,360.000000,1.0,3819.000000,0.000000,1.0,1,1348.443140,0.288889,0,0.906667,0.941281
1,1.0,0.0,1.000000,1.0,0.0,117.000000,360.000000,1.0,6177.000000,0.000000,1.0,1,2787.298488,0.325000,0,0.803333,0.877542
2,1.0,1.0,2.419781,1.0,0.0,182.901096,334.813149,1.0,7142.630796,0.419781,0.0,0,1525.048970,0.546278,0,0.163333,0.059495
3,1.0,1.0,0.000000,1.0,0.0,143.701687,228.983805,0.0,7818.452295,0.000000,0.0,0,2960.178157,0.627563,0,0.150000,0.038194
4,1.0,1.0,0.000000,1.0,0.0,100.000000,360.000000,1.0,3539.000000,0.000000,1.0,1,1252.452100,0.277778,0,0.826667,0.935005


In [21]:
fin_df.to_csv("../model_preds/preds.csv", index=False)